In [5]:
import numpy as np
import pandas as pd
import json 

In [6]:
data=pd.read_json(r"C:\Users\josia\Documents\Shoppe Challenge 1\contacts.json")
data=data.to_numpy()
data=data.tolist()

In [7]:
# Create Object class
class User(object):
    
    def __init__(self,attributes):
        self.dic_index=attributes[0]
        self.ID=[attributes[0]] 
        self.Email=[attributes[1]] if attributes[1] != "" else []
        self.Phone=[attributes[2]] if attributes[2] != "" else []
        self.Contact=attributes[3]
        self.OrderID=[attributes[4]] if attributes[4] !="" else [] 
    
    def get_dic_index(self):
        return self.dic_index
        
    def get_ID(self):
        return self.ID

    def get_Email(self):
        return self.Email

    def get_Phone(self):
        return self.Phone

    def get_Contact(self):
        return self.Contact
    
    def get_OrderID(self):
        return self.OrderID
        
    def matching_attributes(self,user1_attribute,user2_attribute):
        matching = False
        for attribute in user2_attribute:
            if attribute in user1_attribute:
                matching = True
                break
        return matching
    
    def join_users(self,other_user):
        self.ID=join_lists(self.ID , other_user.get_ID())
        self.Email = join_lists(self.Email , other_user.get_Email())
        self.Phone = join_lists(self.Phone , other_user.get_Phone())
        self.Contact= self.Contact + other_user.get_Contact()
        self.Order_ID= join_lists(self.OrderID ,other_user.get_OrderID())
            
    def transform(self):
        #print("transform {}".format(self.dic_index))
        seperator_1="-"
        ticket_trace=""
        IDs= sorted(self.ID)
        for ID in IDs:
            if ticket_trace=="":
                ticket_trace=str(ID)
            else:
                ticket_trace=seperator_1.join((ticket_trace, str(ID)))
        seperator_2=", "
        ticket_trace_contact= seperator_2.join((ticket_trace, str(self.Contact)))
        table=[]
        for ID in self.ID:
            table.append([int(ID),ticket_trace_contact])
        table=np.array(table,ndmin=2)
        #print(table)
        return table
        
    def add_Contact(self, ticket_contact):
        self.Contact+=ticket_contact


def join_lists(list1, list2):
    final_list=list1
    for item in list2:
        if item not in final_list:
            final_list.append(item)
    return final_list


In [8]:
# Common functions
def get_user_email(user):
    return user.get_Email()

def get_user_phone(user):
    return user.get_Phone()

def get_user_OrderID(user):
    return user.get_OrderID()

In [9]:
# Create Comparing fuction

def Contact_Tracing(dataset):
    
    users={}
    Emails={}
    Phones={}
    OrderID={}
    
    for ticket in dataset:
        
        new_user=User(ticket)
        merged=[]
        dic_index=new_user.get_dic_index()
        
        def compare_attribute(attribute_dic,attribute_fn):
            #print("before {}, {},{}".format(Emails,Phones,OrderID))
            attribute_lst=attribute_fn(new_user)
            
            if attribute_lst!=[]:
                
                attribute=attribute_lst[0]
                
                if attribute in attribute_dic:
                    match=attribute_dic[attribute]
                    if match not in merged:
                        merged.append(match)
                        new_user.join_users(users[match])
                        #print("NEW_user {}, {}, {},{}".format(new_user.get_dic_index(),new_user.get_Email(),new_user.get_Phone(),new_user.get_OrderID()))
                        del users[match]
                    
        
        #print("dic index {}".format(dic_index))
        compare_attribute(Emails,get_user_email)
        compare_attribute(Phones,get_user_phone)
        compare_attribute(OrderID,get_user_OrderID)
        #print("user {}, {}, {},{}".format(new_user.get_dic_index(),new_user.get_Email(),new_user.get_Phone(),new_user.get_OrderID()))
        
        for email in new_user.get_Email():
            Emails[email]=dic_index
        for phone in new_user.get_Phone():
            Phones[phone]=dic_index
        for orderid in new_user.get_OrderID():
            OrderID[orderid]=dic_index
        users[dic_index]=new_user
            
    table=[]
    
    for dic_index in users:
        users_info=users[dic_index].transform()
        for info in users_info:
            table.append(info)

    return table

In [10]:
%%time
Compiled_data=Contact_Tracing(data)

Wall time: 8.89 s


In [11]:
Final_Data= pd.DataFrame(Compiled_data, columns = ['ticket_id','ticket_trace/contact'])
Final_Data.ticket_id=Final_Data.ticket_id.astype(int)
Final_Data=Final_Data.sort_values(by='ticket_id')
Final_Data.to_csv(r"C:\Users\josia\Documents\Shoppe Challenge 1\results.csv",index=False)